In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://hadoop-master:7077") \
        .appName("DE-1-4-SparkSession")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.executor.instances", 7) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

24/03/06 09:38:18 WARN Utils: Your hostname, group4-0 resolves to a loopback address: 127.0.0.1; using 192.168.2.91 instead (on interface ens3)
24/03/06 09:38:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/06 09:38:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
reddit_df = spark_session.read.json("hdfs://localhost:9000/user/ubuntu/corpus-webis-tldr-17.json")
reddit_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [4]:
path_pos = "./opinion-lexicon-English/positive-words.txt"
path_neg = "./opinion-lexicon-English/negative-words.txt"
positive_words = set()
negative_words = set()
with open(path_pos, 'r', encoding='utf-8') as file:
    for line in file:
        positive_words.add(line.strip())
with open(path_neg, 'r', encoding='utf-8') as file:
    for line in file:
        negative_words.add(line.strip())

In [5]:
reddit_df = reddit_df.drop("author", "body", "normalizedBody", "id", "subreddit_id", "title")
reddit_df.printSchema()


root
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)



In [6]:
sampled_reddit_df = reddit_df.sample(False, 0.05)
sampled_reddit_df = sampled_reddit_df.filter(sampled_reddit_df["subreddit"] != "NULL")

In [9]:
broadcast_positive_words = spark_session.sparkContext.broadcast(positive_words)
broadcast_negative_words = spark_session.sparkContext.broadcast(negative_words)

In [10]:
import re
def pre_process(summary):
    line = summary.lower()
    line = re.sub(r"[.,]",'',line).split(" ")
    positive_count = sum([word in broadcast_positive_words.value for word in line])
    negative_count = sum([word in broadcast_negative_words.value for word in line])
    if positive_count > negative_count:
        return (1,positive_count,negative_count, 1)
    elif negative_count > positive_count:
        return (-1,positive_count,negative_count, 1)
    else:
        return (0, positive_count, negative_count, 1)
    

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, StructType, StructField

schema = StructType([
    StructField("sentiment", IntegerType(), False),
    StructField("positive_count", IntegerType(), False),
    StructField("negative_count", IntegerType(), False),
    StructField("number_of_tweets", IntegerType(), False)
])

root
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- conotation: struct (nullable = true)
 |    |-- sentiment: integer (nullable = false)
 |    |-- positive_count: integer (nullable = false)
 |    |-- negative_count: integer (nullable = false)
 |    |-- number_of_tweets: integer (nullable = false)



Row(sentiment=1, positive_count=1, negative_count=0, number_of_tweets=1)


24/03/06 09:48:13 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_6_python !
24/03/06 09:48:13 WARN BlockManagerMasterEndpoint: No more replicas available for broadcast_7_python !


In [29]:
from operator import add
import time as time
start_time = time.time()
conotation_udf = udf(pre_process, schema)
reddit_df_conotation = sampled_reddit_df.withColumn("conotation", conotation_udf("summary"))
grouped = reddit_df_conotation.rdd.map(lambda x: (x["subreddit"], (x["conotation"]["sentiment"], x["conotation"]["number_of_tweets"])))
grouped = grouped.reduceByKey(lambda a,b: (a[0] + b[0], a[1] + b[1]))
grouped.take(1)
end_time = time.time()
print(f"Total time: {end_time-start_time} seconds")

[('AskReddit', (1, 1)),
 ('YouShouldKnow', (1, 1)),
 ('atheism', (0, 1)),
 ('todayilearned', (0, 1)),
 ('festivals', (-1, 1)),
 ('AskReddit', (1, 1)),
 ('zelda', (-1, 1)),
 ('Paleo', (0, 1)),
 ('AskReddit', (0, 1)),
 ('gallifrey', (1, 1))]

In [31]:
grouped = grouped.sortBy(lambda x: x[1], ascending = False)
grouped.take(10)

[('leagueoflegends', (195, 5538)),
 ('buildapc', (193, 718)),
 ('summonerschool', (124, 710)),
 ('personalfinance', (91, 719)),
 ('magicTCG', (91, 516)),
 ('photography', (91, 278)),
 ('malefashionadvice', (82, 275)),
 ('wow', (74, 656)),
 ('seduction', (73, 434)),
 ('DotA2', (72, 1121))]

In [33]:
grouped = grouped.sortBy(lambda x: x[1], ascending = True)
grouped.take(10)

[('AskReddit', (-6043, 29300)),
 ('relationships', (-3739, 17765)),
 ('tifu', (-907, 2660)),
 ('relationship_advice', (-374, 2542)),
 ('trees', (-335, 2373)),
 ('politics', (-314, 1831)),
 ('atheism', (-299, 2174)),
 ('pics', (-294, 1764)),
 ('WTF', (-287, 1267)),
 ('funny', (-264, 2002))]

In [38]:
grouped = grouped.filter(lambda x: x[1][1] > 50)
grouped = grouped.sortBy(lambda x: x[1][1], ascending = True)
grouped.take(10)

[('LiverpoolFC', (13, 51)),
 ('Surface', (11, 51)),
 ('gamecollecting', (3, 51)),
 ('law', (1, 51)),
 ('italy', (0, 51)),
 ('4chan', (-7, 51)),
 ('snowboarding', (14, 52)),
 ('aspergers', (6, 52)),
 ('martialarts', (5, 52)),
 ('Buddhism', (2, 52))]